In [12]:
import numpy as np
from sparse_vector import *
from vector_polynomial_isomorphism import *

We note that we are using the base field `QQ` for all computations currently

In [13]:
# Let us initialize constants and vector spaces.

N = 12
d = 3
indeterminates = [var("".join(["x_", str(i), str(j), str(k)])) for i in range(2) for j in range(2) for k in range(3)]
R = PolynomialRing(QQ, indeterminates)
R.inject_variables()

glN = MatrixSpace(QQ, N)
dimV = binomial(N + d - 1, d)
V = VectorSpace(QQ, dimV)

vp_iso = vector_polynomial_isomorphism(R, V, indeterminates, d)

Defining x_000, x_001, x_002, x_010, x_011, x_012, x_100, x_101, x_102, x_110, x_111, x_112


In [14]:
# construct f

M_4by3 = MatrixSpace(R, 4, 3)

x_sub = lambda i,j,k: R(var("".join(["x_", str(i), str(j), str(k)])))

A_12 = M_4by3(0)

for i in range(2):
    for j in range(2):
        for k in range(3):
            A_12[2*i + j, k] = x_sub(i,j,k)

f = A_12.minors(3)

f_vec = [vp_iso.polynomial_to_vector(fi) for fi in f]

In [15]:
# define W = span{f}, W_perp = orthogonal complement of span{f}

W = V.subspace(f_vec)

f_vec_basis = W.basis() # take only the linearly independent ones
    
grad_f = [vp_iso.vector_to_polynomial(fi).gradient() for fi in f_vec_basis]

W_perp = W.complement()

print(W)
print(W_perp)

g = W_perp.basis()
p = W.dimension()
q = W_perp.dimension()

Vector space of degree 364 and dimension 4 over Rational Field
Basis matrix:
4 x 364 dense matrix over Rational Field
Vector space of degree 364 and dimension 360 over Rational Field
Basis matrix:
360 x 364 dense matrix over Rational Field


In [16]:
# action of basis matrix E_ab in glN onto homogeneous polynomial f[i]
def E_action(a, b, i):
    sum = -1*indeterminates[b]*grad_f[i][a]
    return R(sum)

We note `M_f` $:= \tilde{M}_f$

In [17]:
# Pre-processing memoization of entries of M_f

# E_action_f[l][i][j] = (E_ij)(f_l)
E_action_f = [[[0 for j in range(N)] for i in range(N)] for l in range(p)]
for l in range(p):
    for i in range(N):
        for j in range(N):
            E_action_f[l][i][j] = sparsify(vp_iso.polynomial_to_vector(E_action(i,j,l)))

# convert g to sparse representation
g_sparse = [sparsify(g[i]) for i in range(q)]

In [18]:
# Construct M_f

M_f = zero_matrix(QQ, p*q, N^2)
for l in range(p):
    for k in range(q):
        for i in range(N):
            for j in range(N):
                M_f[(l*q) + k,(N*i) + j] = sparse_inner_product(E_action_f[l][i][j], g_sparse[k])

In [19]:
print('rank(M_f) =', M_f.rank())
print('dim(ker(M_f)) =', glN.dimension() - M_f.rank())

rank(M_f) = 120
dim(ker(M_f)) = 24


In [20]:
print((2^2-1) + (2^2-1) + (3^2-1) +1)

15
